In [41]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.orm import Session

import datetime
import pandas as pd
import os

In [2]:
names_basic_df = pd.read_csv('../exploration/imdb_data/name.basics.tsv.gz',
                            delimiter='\t', low_memory=False)

names_basic_df.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0043044,tt0050419,tt0053137,tt0072308"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0038355,tt0037382,tt0117057,tt0071877"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,producer","tt0057345,tt0054452,tt0049189,tt0059956"
3,nm0000004,John Belushi,1949,1982,"actor,writer,soundtrack","tt0077975,tt0080455,tt0078723,tt0072562"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050986,tt0083922,tt0060827,tt0050976"


In [3]:
len(names_basic_df)

9097025

In [4]:
def fill_empty_years(x):
    try:
        return int(x)
    except ValueError:
        return 9999

def fill_empty_strings(x):
    return str(x)

In [5]:
names_basic_df['primaryName'] = names_basic_df['primaryName'].apply(fill_empty_strings)
names_basic_df['birthYear'] = names_basic_df['birthYear'].apply(fill_empty_years)
names_basic_df['deathYear'] = names_basic_df['deathYear'].apply(fill_empty_years)
names_basic_df['primaryProfession'] = names_basic_df['primaryProfession'].apply(fill_empty_strings)
names_basic_df['knownForTitles'] = names_basic_df['knownForTitles'].apply(fill_empty_strings)
names_basic_df.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0043044,tt0050419,tt0053137,tt0072308"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0038355,tt0037382,tt0117057,tt0071877"
2,nm0000003,Brigitte Bardot,1934,9999,"actress,soundtrack,producer","tt0057345,tt0054452,tt0049189,tt0059956"
3,nm0000004,John Belushi,1949,1982,"actor,writer,soundtrack","tt0077975,tt0080455,tt0078723,tt0072562"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050986,tt0083922,tt0060827,tt0050976"


In [43]:
# CREATE DATABASE connection
password = os.environ['AWS_IMDB_PW']
user = 'masterblaster'
endpoint = 'imdb-explorer.clhfspuaimbp.us-east-1.rds.amazonaws.com'
args = "ssl_ca=config/rds-ca-2015-us-east-1-root.pem"

rds_connection_string = f"{user}:{password}@{endpoint}/imdbtest_db2?{args}"
engine = create_engine(f'mysql://{rds_connection_string}')

conn = engine.connect()

In [28]:
Base = declarative_base()
session = Session(bind=engine)

# Object relational mapping for our table
class NamesBasic(Base):
    __tablename__ = 'names_basic'
    id = Column(Integer, primary_key=True)
    nconst = Column(String(50))
    primaryName = Column(String(255))
    birthYear = Column(Integer)
    deathYear = Column(Integer)
    primaryProfession = Column(String(255))
    knownForTitles = Column(String(255))
    

In [14]:
c = 0
maxRows = 100
start_time = datetime.datetime.now()
for index, row in names_basic_df.iterrows():
    if c % 10 == 0:
        print(f'Writing row {c} to DB...')
    record = NamesBasic()
    record.nconst = row['nconst']
    record.primaryName = row['primaryName']
    record.birthYear = row['birthYear']
    record.deathYear = row['deathYear']
    record.primaryProfession = row['primaryProfession']
    record.knownForTitles = row['knownForTitles']
    session.add(record)
    session.commit()
    c = c + 1
    if c >= maxRows: break

end_time = datetime.datetime.now()
print(end_time - start_time)

Writing row 0 to DB...
Writing row 10 to DB...
Writing row 20 to DB...
Writing row 30 to DB...
Writing row 40 to DB...
Writing row 50 to DB...
Writing row 60 to DB...
Writing row 70 to DB...
Writing row 80 to DB...
Writing row 90 to DB...
0:00:23.601200
